In [1]:
import numpy as np

In [4]:
def createDataSet():
    group = np.array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]])
    labels = ['A','A','B','B']
    return group, labels

createDataSet()

(array([[1. , 1.1],
        [1. , 1. ],
        [0. , 0. ],
        [0. , 0.1]]), ['A', 'A', 'B', 'B'])

In [10]:
group = createDataSet()[0].shape[0]
print(group)

4


此处用来解释下面一些numpy函数的作用。
我使用了这个网址用来简化latex符号的编写，很方便。
[在线制作latex数学符号](http://latex.codecogs.com/eqneditor/editor.php)
### tile函数
tile函数是用来重复输出，比如下面的欧式距离公式
$$\sqrt{\sum_{i=1}^{n}\left ( x_{i}-y_{i} \right )^{2} }$$

我们的输入向量现在只有一个，所以要扩充输入向量使其纬度与样本矩阵相同，然后矩阵相减。

一般来说输入向量的列数与样本矩阵相同。

如输入矩阵: $$\begin{bmatrix}
 1& 2\\ 
\end{bmatrix}$$

样本矩阵: $$\begin{bmatrix}
1 & 2\\ 
3 &4 \\ 
5 & 6\\ 
7 & 8
\end{bmatrix}$$


In [ ]:
def classify0(inx, dataSet, labels, k):
    dataSetSize = dataSet.shape[0]  # 得到样本数量
    diffMat = tile(inx, (dataSetSize, 1)) - dataSet  # 构造欧氏距离矩阵
    